In [108]:
import pandas as pd
import numpy as np

In [109]:
d = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

In [110]:
df = d[0]

In [111]:
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Question: Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned   
Answer : Drop rows where Borough value is Not assigned

In [112]:
df.drop(df[df.Borough=='Not assigned'].index,axis=0, inplace=True)

In [113]:
df.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Question: More than one neighborhood can exist in one postal code area.These two rows will be combined into one row with the neighborhoods separated with a comma   
Answer : Merging values of same postal code & join the Neighborhood with commas

In [114]:
df.set_index(['Postcode','Borough'],inplace=True)

In [115]:
df = df.groupby(level=['Postcode','Borough'], sort=False).agg( ','.join)

Question: If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.   

Answer: Check how many rows are there with Neighborhood value as Not assigned

In [116]:
df[df.Neighborhood=='Not assigned']

,,Neighborhood
Postcode,Borough,
M9A,Queen's Park,Not assigned


As there is only one value, we can easily replace it with Borough value by replace mathod

In [117]:
df['Neighborhood'] = df['Neighborhood'].replace(to_replace='Not assigned',value= "Queen's Park")

In [118]:
# check whether there are other not assigned values in Neighborhood column
df[df.Neighborhood=='Not assigned']

,,Neighborhood
Postcode,Borough,


In [120]:
df.head()

,,Neighborhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Downtown Toronto,Queen's Park


Question : In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [121]:
# print the final shape of the dataframe
df.shape

(103, 1)